In [5]:
import API_flightapi as API
import pandas as pd
import json
from pprint import pprint

import DB_Mongo

io = open("requetes/onewaytrip/Requete-du-19-09-2022.json","r")
data = json.load(io)

def billet_id_split(code):
    temp = code.split(':')
    result = temp[1] + '-' + temp[2]
    result = result.split('-0')[0]
    
    return result

def insertion_mongo(data, db):
    
    nbRequetes = len(data["requetes"])

    for i in range (nbRequetes):
        dict_vol = {}
        list_id = []
        list_dt = []
        list_at = []
        list_dur = []
        list_dac = []
        list_aac = []
        list_ac = []
        list_depdate = []
        list_arrdate = []
        dict_price = {}
        list_id_price = []
        list_tot = []
        list_AmountPA = []
        list_AmountPC = []
        list_AmountPI = []
        
        dict_airport = {}
        list_airport_codes = []
        list_airport_names = []
        
        dict_airline = {}
        list_airline_codes = []
        list_airline_names = []
        test = []
        

        #Creation du dictionnaire Vol : 
        for element in (data["requetes"][i]["legs"]):
            list_id.append(billet_id_split(element["id"]))
            list_dt.append(element["departureTime"] + ':00')
            list_at.append(element["arrivalTime"] + ':00')
            list_depdate.append(element['departureDateTime'].split('T')[0])
            list_arrdate.append(element['arrivalDateTime'].split('T')[0])
            test.append((element['departureDateTime'].split('T')[1]).split('.')[0])
            list_dur.append(element["duration"])
            list_dac.append(element["departureAirportCode"])
            list_aac.append(element["arrivalAirportCode"])
            list_ac.append(element["airlineCodes"])
        
                
        dict_vol["id"] = list_id
        dict_vol["departureTime"] = list_dt
        dict_vol["arrivalTime"] = list_at 
        dict_vol["duration"] = list_dur 
        dict_vol["departureAirportCode"] = list_dac 
        dict_vol["arrivalAirportCode"] = list_aac 
        dict_vol["airlineCodes"] = list_ac
        dict_vol["departureDate"] = list_depdate
        dict_vol["arrivalDate"] = list_arrdate   
        
        print(dict_vol["departureTime"])
        print(dict_vol["arrivalTime"])
        
        #Creation du dictionnaire Price
        
        for element in (data["requetes"][i]["fares"]):
            list_id_price.append(element['tripId'].split(':')[1])
            list_tot.append(element['price']['totalAmount'])
            list_AmountPA.append(element['price']['amountPerAdult'])
            list_AmountPC.append(element['price']['amountPerChild']) 
            list_AmountPI.append(element['price']['amountPerInfant'])
        
        dict_price["id"] = list_id_price
        dict_price["totalAmount"] = list_tot
        dict_price["amountPerAdult"] = list_AmountPA
        dict_price["amountPerChild"] = list_AmountPC
        dict_price["amountPerInfant"] = list_AmountPI
        
        #Insertion dans Airports si il n'est pas déja présent
        
        #for element in (data["requetes"][i]["airports"]):
        #    db.insert({element["code"]:element["name"]}, 'Airport')
            
        dict_airport["Nom"] = list_airport_names
        dict_airport["Code"] = list_airport_codes
        
        #Insertion dans Airlines si il n'est pas déja présent
        #for element in (data["requetes"][i]["airlines"]):
        #    db.insert({element["code"]:element["name"]}, 'Airline')
        
        #Insertion dans la base MongoDB
            
        #db.insert(dict_vol, 'Vol')
        #db.insert(dict_price, 'Price')
        
        

        
            
        
            
            
db = DB_Mongo.DB_Mongo()  
insertion_mongo(data, db)


['07:00:00', '09:45:00', '19:40:00', '15:25:00', '06:30:00', '18:20:00', '07:30:00', '11:45:00', '09:40:00', '15:45:00', '17:50:00', '09:40:00', '10:15:00', '07:10:00', '14:55:00', '20:20:00', '15:25:00', '16:50:00', '17:05:00', '11:20:00', '19:40:00', '11:30:00', '20:25:00', '07:05:00', '11:40:00', '20:05:00', '10:15:00', '12:50:00', '20:25:00', '09:45:00', '08:40:00', '13:20:00', '10:20:00', '08:05:00', '10:20:00', '20:35:00', '17:05:00', '09:50:00', '17:50:00', '10:05:00', '15:20:00', '12:05:00', '19:40:00', '14:45:00', '09:25:00', '08:30:00', '10:15:00', '09:10:00', '07:30:00', '17:50:00', '10:20:00', '09:40:00', '06:30:00', '12:40:00', '07:10:00', '10:05:00', '07:25:00', '16:45:00', '18:50:00', '14:25:00', '07:25:00', '07:20:00', '19:25:00', '11:20:00', '12:50:00', '06:10:00', '06:20:00', '15:05:00', '11:40:00', '22:00:00', '15:45:00', '17:05:00', '07:30:00', '07:25:00', '17:45:00', '09:10:00', '09:45:00', '06:30:00', '08:25:00', '09:15:00', '09:35:00', '06:40:00', '10:20:00', '16